In [1]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from torch_geometric.datasets import Actor

/home/litianyue/anaconda3/envs/GNN/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = Actor(root='/tmp/actor')
graph = dataset[0]
print(graph)
print("Length of dataset:",len(dataset)) # 1
print("NumNodes: ",graph.x.shape[0])
print("NumEdges: ",graph.edge_index.shape[1])
print("NumFeats: ",dataset.num_node_features)
print("NumClasses: ",dataset.num_classes)
print("NumTrainingSamples: ",graph.train_mask.nonzero().squeeze().numpy().shape[0])
print("NumValidationSamples: ", graph.val_mask.nonzero().squeeze().numpy().shape[0])
print("NumTestSamples: ", graph.test_mask.nonzero().squeeze().numpy().shape[0])


Data(x=[7600, 932], edge_index=[2, 30019], y=[7600], train_mask=[7600, 10], val_mask=[7600, 10], test_mask=[7600, 10])
Length of dataset: 1
NumNodes:  7600
NumEdges:  30019
NumFeats:  932
NumClasses:  5
NumTrainingSamples:  36480
NumValidationSamples:  24320
NumTestSamples:  15200


In [5]:
# actor_feat
node_feats = sparse.csr_matrix(graph.x.numpy()) # (7600, 932)
# actor_class
node_class = graph.y.numpy() # (7600,)
# actor_edge
edge = graph.edge_index.T.numpy() # (30019, 2)
# actor only has 1 single connected graph
node_list = np.ones((1, graph.x.shape[0]))  # (1, 7600)
edge_list = np.ones((1, graph.edge_index.shape[1]))  # (1, 30019)
data = {
    "node_feats": node_feats,
    "node_class": node_class,
    "edge": edge,
    "node_list": node_list,
    "edge_list": edge_list
}

We can save the data file in only one file `actor.npz`.

In [6]:
np.savez_compressed("actor.npz", **data)
actor = np.load("actor.npz", allow_pickle=True)
actor.files

['node_feats', 'node_class', 'edge', 'node_list', 'edge_list']

In [7]:
!du actor.npz -h

148K	actor.npz


In [8]:
print("Graph is undirected?: ",graph.is_undirected())

Graph is undirected?:  False


## Task

In [9]:
print(graph.train_mask.shape)

tensor([False,  True,  True,  True,  True,  True, False,  True, False, False])
torch.Size([7600, 10])
torch.Size([7600])


In [10]:
train_set = []
val_set = []
test_set = []
for i in range(graph.train_mask.shape[1]):
    train_set.append(graph.train_mask[:,i].nonzero().squeeze().numpy())
    val_set.append(graph.val_mask[:,i].nonzero().squeeze().numpy())
    test_set.append(graph.test_mask[:,i].nonzero().squeeze().numpy())

In [14]:
task_data = {}
for i in range(graph.train_mask.shape[1]):
    task_data['train_'+str(i)] = train_set[i]
    task_data['val_'+str(i)] = val_set[i]
    task_data['test_'+str(i)] = test_set[i]

np.savez_compressed("actor_task.npz", **task_data)

(3648,)


In [ ]:
# task_data['train_0']